In [ ]:
import pandas as pd
import numpy as np
import os
os.chdir('/content/drive/MyDrive/전처리 데이터 ')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/전처리 데이터 '

In [ ]:
os.listdir()

['최종_서울시_흡연구역.csv',
 '___서울시_어린이보호구역_어린이집_유치원.csv',
 '__서울특별시_초_중_고.csv',
 '__거리_50m_이내_구역.csv',
 '서울시 지하철호선별 역별 승하차 인원 정보.csv',
 'BUS_STATION_BOARDING_MONTH_202404.csv',
 'bus_info.xlsx',
 'df 통합.ipynb',
 '서울시 버스정류소 위치정보.csv',
 '서울시 학원 교습소정보.csv',
 '자치구별_흡연구역',
 '음식점_구_위도_경도.csv',
 '서울시 단란주점영업 인허가 정보.csv',
 '특정구_단란주점_정보.csv',
 '특정구_금연구역.csv',
 '특정구_흡연구역.csv',
 '어린이집_초중고_전체데이터.csv',
 '버스정류장_전처리.csv',
 '특정구_버스정류장.csv',
 '.ipynb_checkpoints',
 'traffic1.ipynb',
 '단란주점.ipynb',
 '특정구_학원.csv',
 'knn 어린이 초중고_흡연구역_맵핑.ipynb']

In [ ]:
어린이집 = pd.read_csv('___서울시_어린이보호구역_어린이집_유치원.csv')
흡연구역 = pd.read_csv('최종_서울시_흡연구역.csv')
초중고 = pd.read_csv('__서울특별시_초_중_고.csv')

어린이집 = 어린이집[['대상시설명', '주소','위도','경도','자치구']]
초중고 = 초중고[['학교명', '소재지도로명주소', '위도', '경도', '자치구']]
어린이집.rename(columns={'대상시설명':'학교명'}, inplace=True)
초중고.rename(columns={'소재지도로명주소':'주소'}, inplace=True)
어린이집_초중고 = pd.concat([어린이집, 초중고])
어린이집_초중고.reset_index(drop=True, inplace=True)


흡연구역_radian = np.radians(흡연구역[['위도', '경도']]).values
어린이집_초중고_radian = np.radians(어린이집_초중고[['위도', '경도']]).values

어린이집_초중고['위경도'] = 어린이집_초중고[['위도', '경도']].apply(lambda x: [x['위도'], x['경도']], axis = 1)
흡연구역['위경도'] = 흡연구역[['위도', '경도']].apply(lambda x: [x['위도'], x['경도']], axis = 1)

<ipython-input-13-96309b33070e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  어린이집.rename(columns={'대상시설명':'학교명'}, inplace=True)


In [ ]:
흡연구역[흡연구역['자치구'].isin(['중랑구', '광진구', '마포구', '영등포구'])].to_csv('특정구_흡연구역.csv', index=False)

In [ ]:
# 흡연구역.to_csv('특정구_흡연구역.csv', index=False)
어린이집_초중고.to_csv('어린이집_초중고_전체데이터.csv',index=False)

In [ ]:
def folium_func(구):
  구_어린이집 = 어린이집[어린이집['자치구'] == 구][['위도', '경도']].values
  구_흡연구역 = 흡연구역.loc[흡연구역['자치구'] == 구, ['위도','경도']].values
  구_초중고 = 초중고.loc[초중고['자치구'] == 구, ['위도','경도']].values
  return 구_어린이집, 구_흡연구역, 구_초중고

In [ ]:
import folium
import requests
import json

# 서울 행정구역 json raw파일
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

흡연구역_자치구 = 흡연구역['자치구'].unique()
흡연구역_자치구

array(['영등포구', '성동구', '중구', '서대문구', '송파구', '은평구', '강북구', '관악구', '구로구',
       '동대문구', '마포구', '서초구', '광진구', '중랑구', '강서구', '성북구', '양천구', '용산구'],
      dtype=object)

In [ ]:
print(어린이집_초중고_radian[0])

[0.65445875 2.21386328]


In [ ]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=1, metric='haversine')

knn.fit(흡연구역_radian)

distance, indices = knn.kneighbors(어린이집_초중고_radian)

res = []
for idx, (dist, index) in enumerate(zip(distance, indices)):
  try:
    res.append({
      '어린이집_초중고': 어린이집_초중고.loc[idx, ['위도', '경도']].values,
      '흡연구역': 흡연구역.loc[index, ['위도', '경도']].values[0],
      '거리': dist[0] * 6371000
  })
  except Exception as e:
    print(e, dist, index)

a = pd.DataFrame(res)
a

,어린이집_초중고,흡연구역,거리
0,"[37.497724, 126.8450222]","[37.5076794, 126.8365095]",1337.659334
1,"[37.48599142, 126.884695]","[37.4921086, 126.8847444]",680.213346
2,"[37.4924091, 126.843062]","[37.5076794, 126.8365095]",1793.674083
3,"[37.48722468, 126.852844]","[37.498931, 126.8670866]",1809.228460
4,"[37.49548271, 126.8453299]","[37.5076794, 126.8365095]",1563.563938
...,...,...,...
2245,"[37.49989836, 127.1076425]","[37.5131008, 127.1034334]",1514.266819
2246,"[37.48422849, 126.8414178]","[37.5076794, 126.8365095]",2643.330943
2247,"[37.55893656, 127.1607703]","[37.5551679, 127.1114168]",4370.618284
2248,"[37.55386803, 126.9618701]","[37.5549352, 126.9684545]",592.437386


In [ ]:
from tqdm import tqdm
res = []
for i in tqdm(a['흡연구역']):
   flag = 흡연구역['위경도'].apply(lambda x: (x == i).all())
   res.append(흡연구역[flag].index[0])
흡 = 흡연구역.iloc[res]
흡.reset_index(drop=True, inplace=True)


어_초 = 어린이집_초중고.iloc[a.index]
어_초.reset_index(drop=True, inplace=True)

거리 = a['거리'].reset_index(drop=True)


흡_어초_거리 = pd.concat([흡, 어_초, 거리], axis = 1)
b = 흡_어초_거리.sort_values('거리')
#흡_어초_거리.set_index('거리').sort_index()
# b.to_csv('거리_50m_이내_구역.csv', index=False)
b[b.iloc[:,0] == '서초구'].sort_values('거리')

100%|██████████| 2250/2250 [00:19<00:00, 115.30it/s]


,자치구,설치위치,주소,위도,경도,시설형태,위경도,학교명,주소,위도,경도,자치구,위경도,거리
466,서초구,방배천복개도로 그린골프장방배동 452-1 앞,서울특별시 서초구 방배천로 40,37.480131,126.982678,개방형,"[37.4801311, 126.9826778]",서정,서울특별시 서초구 방배동 455-11,37.480767,126.982787,서초구,"[37.480767, 126.982787]",71.362390
2222,서초구,서초동 1500-6 꽃마을1소공원 남측 부지 내,서울특별시 서초구 꽃마을1길 18,37.473293,127.031210,폐쇄형,"[37.4732933, 127.0312101]",서울양재초등학교,서울특별시 서초구 바우뫼로18길 14,37.473810,127.031886,서초구,"[37.47381019, 127.0318861]",82.838657
463,서초구,방배동 438-38(서울시공공자전거 수리센터 앞),서울특별시 서초구 방배천로 72,37.483171,126.984271,개방형,"[37.483171, 126.9842713]",서초구립예아,서울특별시 서초구 방배동 436-5,37.482379,126.984571,서초구,"[37.482379, 126.984571]",91.951125
1185,서초구,서초구청 주차장 자판기 옆,서울특별시 서초구 남부순환로 2584,37.483743,127.032587,폐쇄형,"[37.4837426, 127.0325868]",양재고등학교,서울특별시 서초구 남부순환로346길 9,37.482932,127.032886,서초구,"[37.48293174, 127.0328864]",93.959058
470,서초구,서초대로 78길 24 서초동 1327-5 스타벅스 앞,서울특별시 서초구 서초대로78길 24,37.495627,127.027721,개방형,"[37.495627, 127.0277213]",샤이닝키즈,서울특별시 서초구 서초동 1326-17,37.495091,127.026805,서초구,"[37.495091, 127.026805]",100.434167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284,서초구,남부순환로 2636 건물 우측 골목(오선채 좌측골목),서울특별시 서초구 남부순환로 2636,37.484865,127.039114,라인형흡연구역,"[37.4848647, 127.039114]",휘문고등학교,서울특별시 강남구 역삼로 541,37.504928,127.062798,강남구,"[37.50492792, 127.0627984]",3056.644832
732,서초구,논현로1길 23 아파트 외부 보도 가각지점,서울특별시 서초구 논현로1길 23,37.471576,127.050471,라인형흡연구역,"[37.4715762, 127.0504708]",푸른유치원,서울특별시 강남구 일원로14길 25,37.484798,127.081101,강남구,"[37.48479809, 127.0811005]",3076.829772
1402,서초구,논현로1길 23 아파트 외부 보도 가각지점,서울특별시 서초구 논현로1길 23,37.471576,127.050471,라인형흡연구역,"[37.4715762, 127.0504708]",중산고등학교,서울특별시 강남구 일원로14길 22,37.485458,127.080717,강남구,"[37.48545837, 127.0807166]",3083.199006
1952,서초구,논현로1길 23 아파트 외부 보도 가각지점,서울특별시 서초구 논현로1길 23,37.471576,127.050471,라인형흡연구역,"[37.4715762, 127.0504708]",중동중학교,서울특별시 강남구 일원로8길 37,37.488801,127.078167,강남구,"[37.48880119, 127.0781671]",3105.041559


In [ ]:
어린이집_초중고 = pd.read_csv('어린이집_초중고_전체데이터.csv')
어린이집_초중고
흡연구역

,자치구,설치위치,주소,위도,경도,시설형태,위경도
0,영등포구,한강프리젠,서울특별시 영등포구 양평로24길 16 (양평동5가),37.538041,126.892061,완전개방형,"[37.5380405, 126.8920611]"
1,영등포구,프리가,서울특별시 영등포구 영등포로 390,37.513310,126.921288,완전개방형,"[37.5133097, 126.9212875]"
2,영등포구,에이스테크노타워,서울특별시 영등포구 당산로2길 12,37.515130,126.895416,완전개방형,"[37.5151303, 126.8954163]"
3,영등포구,고려빌딩,서울특별시 영등포구 여의도동 여의대방로67길 8,37.519543,126.931100,개방형,"[37.5195427, 126.9310998]"
4,영등포구,리버타워오피스텔,서울특별시 영등포구 버드나루로 50 (영등포동2가),37.520609,126.912195,개방형,"[37.5206095, 126.9121946]"
...,...,...,...,...,...,...,...
1019,용산구,용산더프라임,서울특별시 용산구 원효로90길 11,37.540210,126.970097,폐쇄형,"[37.5402095, 126.9700966]"
1020,용산구,한국폴리텍대학,서울특별시 용산구 보광로 73,37.529138,126.996538,개방형,"[37.5291378, 126.9965384]"
1021,용산구,한국폴리텍대학,서울특별시 용산구 보광로 73,37.529138,126.996538,개방형,"[37.5291378, 126.9965384]"
1022,용산구,한국폴리텍대학,서울특별시 용산구 보광로 73,37.529138,126.996538,개방형,"[37.5291378, 126.9965384]"


In [ ]:
흡연구역_gdf_to_crs

,자치구,설치위치,주소,위도,경도,시설형태,위경도,geometry
0,영등포구,한강프리젠,서울특별시 영등포구 양평로24길 16 (양평동5가),37.538041,126.892061,완전개방형,"[37.5380405, 126.8920611]",POINT (190460.297 548731.614)
1,영등포구,프리가,서울특별시 영등포구 영등포로 390,37.513310,126.921288,완전개방형,"[37.5133097, 126.9212875]",POINT (193041.047 545984.240)
2,영등포구,에이스테크노타워,서울특별시 영등포구 당산로2길 12,37.515130,126.895416,완전개방형,"[37.5151303, 126.8954163]",POINT (190754.005 546188.532)
3,영등포구,고려빌딩,서울특별시 영등포구 여의도동 여의대방로67길 8,37.519543,126.931100,개방형,"[37.5195427, 126.9310998]",POINT (193909.057 546675.344)
4,영등포구,리버타워오피스텔,서울특별시 영등포구 버드나루로 50 (영등포동2가),37.520609,126.912195,개방형,"[37.5206095, 126.9121946]",POINT (192237.902 546795.137)
...,...,...,...,...,...,...,...,...
1019,용산구,용산더프라임,서울특별시 용산구 원효로90길 11,37.540210,126.970097,폐쇄형,"[37.5402095, 126.9700966]",POINT (197357.197 548967.292)
1020,용산구,한국폴리텍대학,서울특별시 용산구 보광로 73,37.529138,126.996538,개방형,"[37.5291378, 126.9965384]",POINT (199694.025 547738.055)
1021,용산구,한국폴리텍대학,서울특별시 용산구 보광로 73,37.529138,126.996538,개방형,"[37.5291378, 126.9965384]",POINT (199694.025 547738.055)
1022,용산구,한국폴리텍대학,서울특별시 용산구 보광로 73,37.529138,126.996538,개방형,"[37.5291378, 126.9965384]",POINT (199694.025 547738.055)


In [ ]:
import geopandas as gpd
from tqdm import tqdm

어린이집_초중고_gdf = gpd.GeoDataFrame(어린이집_초중고, geometry=gpd.points_from_xy(어린이집_초중고.경도, 어린이집_초중고.위도), crs='epsg:4326')
흡연구역_gdf = gpd.GeoDataFrame(흡연구역, geometry=gpd.points_from_xy(흡연구역.경도, 흡연구역.위도), crs='epsg:4326')
흡연구역_gdf_to_crs = 흡연구역_gdf.to_crs(epsg=5186)
res = []

for idx, i in enumerate(tqdm(흡연구역_gdf_to_crs['geometry'])):
  data = 어린이집_초중고_gdf.to_crs(epsg=5179).geometry.distance(흡연구역_gdf.to_crs(epsg=5179).iloc[idx].geometry)
  res.append([idx, data.argmin(), data.min()])

100%|██████████| 1024/1024 [00:20<00:00, 49.27it/s]


In [ ]:
q = pd.DataFrame(sorted(res, key = lambda x: x[2])[:20])

w = 흡연구역_gdf.iloc[q.iloc[:,0]]
w['거리'] = q.iloc[:,2].values
w

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,자치구,설치위치,주소,위도,경도,시설형태,위경도,geometry,거리
896,중랑구,중곡 스포렉스,서울특별시 중랑구 면목로23길 20 (면목동),37.574357,127.083639,NaN,"[37.5743574, 127.0836385]",POINT (127.08364 37.57436),9.544182
571,광진구,허리우드(8대),"서울특별시 광진구 자양번영로 33 (자양동, 성보빌딩)",37.532669,127.075258,완전폐쇄형,"[37.5326691, 127.075258]",POINT (127.07526 37.53267),23.165210
502,광진구,다원,"서울특별시 광진구 능동로 391, 지층 (중곡동)",37.563544,127.088830,완전폐쇄형,"[37.563544, 127.0888304]",POINT (127.08883 37.56354),31.060681
534,광진구,준당구장,"서울특별시 광진구 능동로 340, 지층 (중곡동)",37.563544,127.088830,완전폐쇄형,"[37.563544, 127.0888304]",POINT (127.08883 37.56354),31.060681
535,광진구,제우스 당구장,"서울특별시 광진구 능동로 378, 지층 (중곡동)",37.563544,127.088830,완전폐쇄형,"[37.563544, 127.0888304]",POINT (127.08883 37.56354),31.060681
672,중랑구,i7 PC방,서울특별시 중랑구 면목동 665-3번지,37.578200,127.085030,NaN,"[37.5781997, 127.0850301]",POINT (127.08503 37.57820),37.299421
307,마포구,이룸PC,서울특별시 마포구 도화4길 37,37.539489,126.951135,폐쇄형,"[37.5394893, 126.9511346]",POINT (126.95113 37.53949),40.280678
787,중랑구,하라보 보드게임장,서울특별시 중랑구 면목로27길 92 지하,37.574266,127.080535,NaN,"[37.5742658, 127.0805347]",POINT (127.08053 37.57427),42.513006
52,영등포구,월드메르디앙비즈센터,서울특별시 영등포구 양산로 53 (양평동3가),37.526911,126.890146,개방형,"[37.5269111, 126.8901455]",POINT (126.89015 37.52691),45.540966
446,광진구,유니버설아트센터,서울특별시 광진구 천호대로 664,37.550395,127.087338,완전 폐쇄형,"[37.5503951, 127.087338]",POINT (127.08734 37.55040),45.693230


In [ ]:
흡연구역_gdf.iloc[np.array(sorted(res, key = lambda x: x[1]))[:,0]]

,자치구,설치위치,주소,위도,경도,시설형태,위경도,geometry
0,영등포구,한강프리젠,서울특별시 영등포구 양평로24길 16 (양평동5가),37.538041,126.892061,완전개방형,"[37.5380405, 126.8920611]",POINT (126.89206 37.53804)
1,영등포구,프리가,서울특별시 영등포구 영등포로 390,37.513310,126.921288,완전개방형,"[37.5133097, 126.9212875]",POINT (126.92129 37.51331)
2,영등포구,에이스테크노타워,서울특별시 영등포구 당산로2길 12,37.515130,126.895416,완전개방형,"[37.5151303, 126.8954163]",POINT (126.89542 37.51513)
3,영등포구,고려빌딩,서울특별시 영등포구 여의도동 여의대방로67길 8,37.519543,126.931100,개방형,"[37.5195427, 126.9310998]",POINT (126.93110 37.51954)
4,영등포구,리버타워오피스텔,서울특별시 영등포구 버드나루로 50 (영등포동2가),37.520609,126.912195,개방형,"[37.5206095, 126.9121946]",POINT (126.91219 37.52061)
...,...,...,...,...,...,...,...,...
1019,용산구,용산더프라임,서울특별시 용산구 원효로90길 11,37.540210,126.970097,폐쇄형,"[37.5402095, 126.9700966]",POINT (126.97010 37.54021)
1020,용산구,한국폴리텍대학,서울특별시 용산구 보광로 73,37.529138,126.996538,개방형,"[37.5291378, 126.9965384]",POINT (126.99654 37.52914)
1021,용산구,한국폴리텍대학,서울특별시 용산구 보광로 73,37.529138,126.996538,개방형,"[37.5291378, 126.9965384]",POINT (126.99654 37.52914)
1022,용산구,한국폴리텍대학,서울특별시 용산구 보광로 73,37.529138,126.996538,개방형,"[37.5291378, 126.9965384]",POINT (126.99654 37.52914)


In [ ]:
흡연구역_gdf.iloc[np.array(sorted(res, key = lambda x: x[1]))[:,0]]

,자치구,설치위치,주소,위도,경도,시설형태,위경도,geometry
171,구로구,고대구로병원,서울특별시 구로구 구로동로 148,37.492109,126.884744,NaN,"[37.4921086, 126.8847444]",POINT (126.88474 37.49211)
918,성북구,성북구보건소 10층 옥상,서울특별시 성북구 화랑로 63 (하월곡동),37.603960,127.038837,완전개방형,"[37.6039598, 127.0388371]",POINT (127.03884 37.60396)
927,성북구,현대백화점 미아점 옥상,서울특별시 성북구 동소문로 316 (하월곡동),37.604645,127.039391,폐쇄형,"[37.6046451, 127.0393912]",POINT (127.03939 37.60465)
919,성북구,성북구청 12층 옥상,서울특별시 성북구 보문로 168 (삼선동5가),37.588540,127.015878,개방형,"[37.5885404, 127.0158778]",POINT (127.01588 37.58854)
920,성북구,성북구청 5층 휴게공간,서울특별시 성북구 보문로 168 (삼선동5가),37.588540,127.015878,개방형,"[37.5885404, 127.0158778]",POINT (127.01588 37.58854)
...,...,...,...,...,...,...,...,...
53,영등포구,맨하탄빌딩,서울특별시 영등포구 국제금융로6길 33,37.520782,126.926916,개방형,"[37.520782, 126.9269161]",POINT (126.92692 37.52078)
123,은평구,서울기록원,서울특별시은평구 통일로62길 7,37.607383,126.934719,폐쇄형,"[37.6073832, 126.9347185]",POINT (126.93472 37.60738)
137,은평구,정원빌딩,서울특별시은평구 진흥로 235,37.610182,126.935001,개방형,"[37.610182, 126.935001]",POINT (126.93500 37.61018)
92,서대문구,1층,서울특별시 서대문구 통일로37길 51,37.586025,126.944290,폐쇄형,"[37.5860245, 126.9442903]",POINT (126.94429 37.58602)


In [ ]:
sorted(res, key = lambda x: x[1])

[[171, 24],
 [918, 58],
 [927, 58],
 [919, 63],
 [920, 63],
 [133, 76],
 [127, 77],
 [128, 77],
 [129, 77],
 [143, 84],
 [144, 84],
 [145, 84],
 [146, 84],
 [132, 89],
 [96, 92],
 [141, 99],
 [136, 101],
 [122, 102],
 [396, 104],
 [473, 104],
 [487, 104],
 [594, 104],
 [614, 104],
 [551, 105],
 [552, 105],
 [553, 105],
 [554, 105],
 [621, 105],
 [398, 106],
 [561, 106],
 [571, 106],
 [572, 106],
 [577, 106],
 [466, 109],
 [471, 109],
 [546, 109],
 [547, 109],
 [549, 109],
 [550, 109],
 [411, 110],
 [652, 110],
 [495, 111],
 [562, 111],
 [563, 111],
 [622, 111],
 [623, 111],
 [595, 112],
 [598, 112],
 [616, 112],
 [617, 112],
 [533, 113],
 [536, 113],
 [545, 113],
 [604, 113],
 [634, 113],
 [540, 114],
 [541, 114],
 [543, 114],
 [602, 114],
 [635, 114],
 [636, 114],
 [637, 114],
 [638, 114],
 [679, 114],
 [503, 115],
 [479, 116],
 [480, 116],
 [481, 116],
 [494, 116],
 [582, 116],
 [583, 116],
 [585, 116],
 [586, 116],
 [588, 116],
 [589, 116],
 [606, 116],
 [607, 116],
 [608, 116],
 [6